In [ ]:
import h5py
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
# Complete file name of the Lattice Microbes Trajectory
# filename = '/home/zane/Models/mincell/mc4d/Data/division_6000_1/MinCell.lm'
# wd = '/Data1/zane/Models/mincell/Dec23_28_Jan2/'
# wd = '/Data1/zane/Models/mincell/Jan31/'
# wd = '/Data1/zane/Models/mincell/Mar10/lm_traj/'
wd = '/Data1/zane/Models/mincell/merged/'

figDir = '/home/zane/Pictures/mc4d/'

# fn = 'MC_'
fn = 'MinCell_'
ext = '.lm'
reps = [x for x in range(1,21)]
for rep in reps:
    traj = h5py.File(wd+fn+str(rep)+ext)
    if float(traj['Simulations']['0000001']['LatticeTimes'][-1])!=6000:
        print(rep, traj['Simulations']['0000001']['LatticeTimes'][-1])
    # reps.append(i)
reps

In [ ]:
# with open(wd+'reps.txt') as f:
#     lines = [line.rstrip() for line in f]
# print(lines)

In [ ]:
sNbin = traj['Parameters']['SpeciesNames']
sN = []
for n in sNbin:
    sN.append(n[0].decode("utf-8"))

def getIdx(name):
    return int(sN.index(name)+1)

In [ ]:
siteNames = traj['Parameters'].attrs['siteTypeNames'].decode("utf-8").split(',')

def getSiteIdx(name):
    return int(siteNames.index(name))

print(getSiteIdx('outer_cytoplasm'))
siteNames

In [ ]:
# np.array(traj['Simulations']['0000001']['Sites']['0000006000'])

In [ ]:
def getcoords(spec, time=None):

    coords = []

    for rep in reps:

        traj = h5py.File(wd+fn+str(rep)+ext)

        if time is None:
            t = int(traj['Simulations']['0000001']['LatticeTimes'][-1])
        else:
            t = time
    
        pL = np.array(traj['Simulations']['0000001']['Lattice']['000000{}'.format(t)])

        scoords = np.argwhere(pL==getIdx(spec))

        scoords = scoords.T[0:3].T

        coords.append(scoords)

        # print(coords)

    return coords

In [ ]:
def getcoordsMult(specList, time=None):

    coords = []

    for rep in reps:

        traj = h5py.File(wd+fn+str(rep)+ext)

        if time is None:
            t = int(traj['Simulations']['0000001']['LatticeTimes'][-1])
        else:
            t = time
    
        pL = np.array(traj['Simulations']['0000001']['Lattice']['000000{}'.format(t)])

        scoords = None

        for spec in specList:

            if scoords is None:

                scoords = np.argwhere(pL==getIdx(spec))

            else:

                sc = np.argwhere(pL==getIdx(spec))

                scoords = np.concatenate((scoords, sc), axis=0)

        scoords = scoords.T[0:3].T

        coords.append(scoords)

        # print(coords)

    return coords

In [ ]:
# def getSiteCoords(site, time=None):

#     coords = []

#     for rep in reps:

#         traj = h5py.File(wd+fn+str(rep)+ext)

#         if time is None:
#             t = int(traj['Simulations']['0000001']['LatticeTimes'][-1])
#         else:
#             t = time
    
#         pL = np.array(traj['Simulations']['0000001']['Sites']['000000{}'.format(t)])

#         scoords = np.argwhere(pL==getSiteIdx(site))

#         scoords = scoords.T[0:3].T

#         coords.append(scoords)

#         # print(coords)

#     return coords

In [ ]:
def getDividedDist(data=None, spec=None, specList=None):

    counts_per_cell = []

    for i in range(len(reps)):
    
        scoords = data[i]
    
        cell1 = 0
        cell2 = 0
    
        for coord in scoords:
    
            if coord[0]<64:
    
                cell1+=1
    
            else:
    
                cell2+=1
    
        counts_per_cell.append(cell1)
        counts_per_cell.append(cell2)

    return counts_per_cell

In [ ]:
def getPartitionedParticles(data):

    countsPerCell = {
        'Left':[],
        'Right':[]
    }

    for i in range(len(reps)):
    
        scoords = data[i]
    
        cell1 = 0
        cell2 = 0
    
        for coord in scoords:
    
            if coord[0]<64:
    
                cell1+=1
    
            else:
    
                cell2+=1
    
        countsPerCell['Left'].append(cell1)
        countsPerCell['Right'].append(cell2)

    return countsPerCell

In [ ]:
ptsgC = getcoords('P_0779')

In [ ]:
dist = getDividedDist(ptsgC)
plt.hist(dist)

In [ ]:
ptsgPart = getPartitionedParticles(ptsgC)
ptsgPart

In [ ]:
x = np.arange(len(reps))  # the label locations
width = 0.15  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for side, count in ptsgPart.items():
    # offset = width * multiplier
    if side == 'Left':
        offset = -width
    elif side == 'Right':
        offset = width
    rects = ax.bar(x + offset, count, width, label=side)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Particle Count')
ax.set_xlabel('Cell Simulation Index')
ax.set_title('PtsG Partitioning After Division')
ax.set_xticks(x, reps)
ax.legend(loc='upper left', ncols=3)
# ax.set_ylim(0, 700)

plt.show()

In [ ]:
riboSpec = ['ribosomeP']
for pID in sN:
    if pID.startswith('RB_'):
        riboSpec.append(pID)
riboCoords = getcoordsMult(riboSpec, time='6300')

In [ ]:
dist = getDividedDist(riboCoords)
plt.hist(dist, color='yellow', edgecolor='k', linewidth=1.2)
plt.axvline(np.average(dist), color='red', linewidth=3)
plt.legend(['Average: '+str(int(np.average(dist)))])
plt.xlabel('Ribosomes')
plt.ylabel('Cells')
plt.title('Ribosome Parititioning to Divided Cells')
plt.savefig(f'{figDir}ribosome_division.png')

In [ ]:
riboPart = getPartitionedParticles(riboCoords)
riboPart

In [ ]:
x = np.arange(len(reps))  # the label locations
width = 0.15  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for side, count in riboPart.items():
    # offset = width * multiplier
    if side == 'Left':
        offset = -width
    elif side == 'Right':
        offset = width
    rects = ax.bar(x + offset, count, width, label=side)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ribosome Count')
ax.set_xlabel('Cell Simulation Index')
ax.set_title('Ribosome Partitioning After Division')
ax.set_xticks(x, reps)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 700)

plt.show()

In [ ]:
# GapD Data
GAPD = 'P_0607'

specCoordsGAPD = getcoords(GAPD, time='6300')

distGAPD = getDividedDist(specCoordsGAPD)

partitionGAPD = getPartitionedParticles(specCoordsGAPD)

# PtsG Data
PTSG = 'P_0779'

specCoordsPTSG = getcoords(PTSG, time='6300')

distPTSG = getDividedDist(specCoordsPTSG)

partitionPTSG = getPartitionedParticles(specCoordsPTSG)

# Ribosome Data
riboID = ['ribosomeP']

for pID in sN:
    if pID.startswith('RB_'):
        riboID.append(pID)

specCoordsRIBO = getcoordsMult(riboID, time='6300')

distRIBO = getDividedDist(specCoordsRIBO)

partitionRIBO = getPartitionedParticles(specCoordsRIBO)

# Degradosome Data
degID = ['Degradosome']

for pID in sN:
    if pID.startswith('D_'):
        degID.append(pID)

specCoordsDEG = getcoordsMult(degID, time='6300')

distDEG = getDividedDist(specCoordsDEG)

partitionDEG = getPartitionedParticles(specCoordsDEG)

In [ ]:
dists = [distRIBO, distDEG, distPTSG, distGAPD]
partitions = [partitionRIBO, partitionDEG, partitionPTSG, partitionGAPD]
labels = ['Ribosomes', 'Degradosomes', 'PtsG', 'GapD']

In [ ]:
fig, axs = plt.subplots(4,2, figsize=(6,8))
plt.rcParams.update({'font.size': 8})
plt.rcParams.update({"font.family": 'sans-serif'})

plt.rcParams['figure.dpi'] = 300

x = np.arange(len(reps))  # the label locations
width = 0.15  # the width of the bars
multiplier = 0

for i in range(len(dists)):

    data = dists[i]

    nbars, nbins, npatches = axs[i,0].hist(data, color='yellow', edgecolor='k', linewidth=1.2)
    axs[i,0].axvline(np.average(data), color='red', linewidth=3)
    axs[i,0].set_ylim(0,np.max(nbars)*1.3)
    axs[i,0].legend(['Avg: '+str(int(np.average(data)))], loc='upper right')
    axs[i,0].set_xlabel('Particles Per Daughter')
    axs[i,0].set_ylabel('Cells')
    axs[i,0].set_title(labels[i])

    data = partitions[i]
    
    ymax = 0
    for side, count in data.items():
        # offset = width * multiplier
        if side == 'Left':
            offset = -width
        elif side == 'Right':
            offset = width
        rects = axs[i,1].bar(x + offset, count, width, label=side)
        if max(count)>ymax:
            ymax=float(max(count))
        # ax.bar_label(rects, padding=3)
        multiplier += 1
    
    # Add some text for labels, title and custom x-axis tick labels, etc.
    axs[i,1].set_ylabel('Particles Per Daughter')
    axs[i,1].set_xlabel('Cell Simulation Index')
    axs[i,1].set_title(labels[i])
    axs[i,1].set_xticks([0,4,9,14,19], [1,5,10,15,20])
    # axs[i,1].legend(loc='upper left', ncols=3)
    axs[i,1].set_ylim(0, ymax*1.1)

plt.tight_layout()
plt.savefig(f'{figDir}partitioning.png')

In [ ]:
# def plotPartitioning(data, specID=None):

#     if specID is None:
#         sID = ''
#     else:
#         sID = specID + ' '

#     x = np.arange(len(reps))  # the label locations
#     width = 0.15  # the width of the bars
#     multiplier = 0
    
#     fig, ax = plt.subplots(layout='constrained')

#     ymax = 0
#     for side, count in data.items():
#         # offset = width * multiplier
#         if side == 'Left':
#             offset = -width
#         elif side == 'Right':
#             offset = width
#         rects = ax.bar(x + offset, count, width, label=side)
#         if max(count)>ymax:
#             ymax=float(max(count))
#         # ax.bar_label(rects, padding=3)
#         multiplier += 1
    
#     # Add some text for labels, title and custom x-axis tick labels, etc.
#     ax.set_ylabel('Particle Count')
#     ax.set_xlabel('Cell Simulation Index')
#     ax.set_title(sID + 'Partitioning at 105 Minutes')
#     ax.set_xticks(x, reps)
#     ax.legend(loc='upper left', ncols=3)
#     ax.set_ylim(0, ymax*1.2)

#     if specID is not None:
#         plt.savefig(f'./partitioning_{specID}.png')
    
#     plt.show()

In [ ]:
# def plotDivDist(data, specID = None):

#     plt.rcParams.update({'font.size': 12})
#     plt.rcParams.update({"font.family": 'sans-serif'})

#     plt.rcParams['figure.dpi'] = 300

#     if specID is None:
#         sID = ''
#     else:
#         sID = specID + ' '

#     plt.hist(data, color='yellow', edgecolor='k', linewidth=1.2)
#     plt.axvline(np.average(data), color='red', linewidth=3)
#     plt.legend(['Average: '+str(int(np.average(dist)))])
#     plt.xlabel('Particle Count')
#     plt.ylabel('Cells')
#     plt.title(sID + ' in Daughter Cells')

#     if specID is not None:
#         plt.savefig(f'./dist_{specID}.png')
    
#     plt.show()

In [ ]:
break

In [ ]:
break

In [ ]:
specID = 'P_0607'

specCoords = getcoords(specID, time='6300')

dist = getDividedDist(specCoords)
plotDivDist(dist, specID)

partition = getPartitionedParticles(specCoords)
plotPartitioning(partition, specID)

In [ ]:
specID = 'outer_cytoplasm'

specCoords = getSiteCoords(specID, time='6300')

dist = getDividedDist(specCoords)
plotDivDist(dist, specID)

partition = getPartitionedParticles(specCoords)
plotPartitioning(partition, specID)

In [ ]:
specID = 'P_0779'

specCoords = getcoords(specID, time='6300')

dist = getDividedDist(specCoords)
plotDivDist(dist, specID)

partition = getPartitionedParticles(specCoords)
plotPartitioning(partition, specID)

In [ ]:
specID = ['ribosomeP']

for pID in sN:
    if pID.startswith('RB_'):
        specID.append(pID)

specCoords = getcoordsMult(specID, time='6300')

dist = getDividedDist(specCoords)
plotDivDist(dist, 'Total Ribosomes')

partition = getPartitionedParticles(specCoords)
plotPartitioning(partition, 'Total Ribosomes')

In [ ]:
specID = ['Degradosome']

for pID in sN:
    if pID.startswith('D_'):
        specID.append(pID)

specCoords = getcoordsMult(specID, time='6300')

dist = getDividedDist(specCoords)
plotDivDist(dist, 'Total Degradosomes')

partition = getPartitionedParticles(specCoords)
plotPartitioning(partition, 'Total Degradosomes')

In [ ]:
break

In [ ]:
specID = ['Degradosome']

wd = '/home/zane/Models/mincell/mc4d/Data/ceil_test/'
fn = 'MinCell_'
reps = [1,2,3,4,5,6,7,8,9,10]
ext = '.lm'

for pID in sN:
    if pID.startswith('D_'):
        specID.append(pID)

specCoords = getcoordsMult(specID, time='0200')

dist = getDividedDist(specCoords)
plotDivDist(dist, 'Total Degradosomes')

partition = getPartitionedParticles(specCoords)
plotPartitioning(partition, 'Total Degradosomes')

In [ ]:
specID = ['ribosomeP']

wd = '/home/zane/Models/mincell/mc4d/Data/dna_shrink_test_32_1/'
fn = 'MinCell_'
reps = [1]
ext = '.lm'

for pID in sN:
    if pID.startswith('RB_'):
        specID.append(pID)

specCoords = getcoordsMult(specID, time='0030')

dist = getDividedDist(specCoords)
plotDivDist(dist, 'Total Degradosomes')

partition = getPartitionedParticles(specCoords)
plotPartitioning(partition, 'Total Degradosomes')

In [ ]:
specID = 'P_0407'

wd = '/home/zane/Models/mincell/mc4d/Data/dna_shrink_test_32_1/'
fn = 'MinCell_'
reps = [1]
ext = '.lm'

specCoords = getcoords(specID, time='0030')

dist = getDividedDist(specCoords)
plotDivDist(dist, 'Total {}'.format(specID))

partition = getPartitionedParticles(specCoords)
plotPartitioning(partition, 'Total {}'.format(specID))

In [ ]:
specID = ['P_0652']

for pID in sN:
    if pID.startswith('S_'):
        specID.append(pID)

specCoords = getcoordsMult(specID)

dist = getDividedDist(specCoords)
plotDivDist(dist, 'Total SecY')

partition = getPartitionedParticles(specCoords)
plotPartitioning(partition, 'Total SecY')

In [ ]:
riboSpec = ['ribosomeP']
for pID in sN:
    if pID.startswith('RB_'):
        riboSpec.append(pID)
riboCoords = getcoordsMult(riboSpec, time=2400)

In [ ]:
dist = []
for coord in riboCoords[0]:
    r = 10*((coord[0]-64)**2+(coord[1]-32)**2+(coord[2]-32)**2)**0.5
    dist.append(r)
bins = [50,100,150,200,250]
dens = []
for b in bins:
    d = 0
    for r in dist:
        if b-50<r<b:
            d+=1
    dens.append(d/(4/3*np.pi*(b**3-(b-50)**3)))
plt.scatter(bins,dens)
plt.xlabel('Radial Position')
plt.ylabel('Ribosome Density (count/V)')

In [ ]:
break

In [ ]:
traj = traj = h5py.File(wd+fn+str(10)+ext)

In [ ]:
traj['Parameters'].attrs['siteTypeNames']

In [ ]:
siteStr = traj['Parameters'].attrs['siteTypeNames'].decode("utf-8")
siteNames = siteStr.split(',')
siteNames

In [ ]:
sLattice = np.array(traj['Simulations']['0000001']['Sites']['0000006000'])
sLattice.shape

In [ ]:
outerCytoplasm = np.argwhere(sLattice == 3)
len(outerCytoplasm)

In [ ]:
specID = ['Degradosome']

for pID in sN:
    if pID.startswith('D_'):
        specID.append(pID)

specCoords = getcoordsMult(specID)

In [ ]:
for coord in specCoords[-2]:
    # print(coord)
    print(sLattice[coord[0],coord[1],coord[2]])

In [ ]:
ptsgID = 'P_0779'

ptsgCoords = getcoords(ptsgID)

for coord in ptsgCoords[-2]:
    # print(coord)
    print(sLattice[coord[0],coord[1],coord[2]])

In [ ]:
break

In [ ]:
# Load in file using h5py since it is in h5 format
traj = h5py.File(wd+fn+str(1)+ext)

In [ ]:
# We can check the main components of the file. SHould include: Model, Parameters, and Simulations
traj.keys()

In [ ]:
# Parameters contiains tables with information about the system
traj['Parameters'].keys()

In [ ]:
# Each part of the simulation has a list of attributes that can be referenced
traj['Parameters'].attrs.keys()

In [ ]:
# To get the list of species names in the trajectory, we need to decode them from binary to utf-8
# I just create a list containing all the species names

# Names that Zane thinks are useful:
# Proteins: P_XXXX where XXXX is the NCBI locus number (JCVISYN3A_XXXX)
# Ribosomes (inactive): ribosomeP
# Ribosomes (active): RB_XXXX (ribosome:mRNA bound state for mRNA from gene XXXX)
# RNA polymerase (inactive): RNAP
# RNA polymerase (active): RP_XXXX (RNAP:gene transcription state) (Do not currently recommend, this has some weird attributes due to the hybrid simulation)

sNbin = traj['Parameters']['SpeciesNames']
sN = []
for n in sNbin:
    sN.append(n[0].decode("utf-8"))
sN

In [ ]:
# We will need to be able to get the indices of particles when searching the lattice in the trajectory
# Function to go from specie name to index
def getIdx(name):
    return int(sN.index(name)+1)

In [ ]:
getIdx('P_0779')

In [ ]:
# Check how many simulations are in the trajectory. Should be a single replicate
traj['Simulations'].keys()

In [ ]:
# The trajectory contains information in multiple tables
traj['Simulations']['0000001'].keys()

In [ ]:
# To get the positions of particles, we need 'Lattice'
# The write times are intervals of 1 second, so the write step indices are also in units of seconds
float(traj['Simulations']['0000001']['LatticeTimes'][-1])==6000

In [ ]:
# We will get the particle lattice at one point in time as a numpy array (division starts somewhere around 1 hr (3600) )
time = 3600 # seconds
pL = np.array(traj['Simulations']['0000001']['Lattice']['000000{}'.format(time)])

In [ ]:
# Dimensions of the lattice are x, y, z, n where n is the number of particles allowed per lattice cube
pL.shape

In [ ]:
# We can get an array of coordinates for each particle of a particular type
# Only need first 3 parts of each coordinate (x,y,z)
pID = 'P_0779'
np.argwhere(pL==getIdx(pID))

In [ ]:
# We can get ribosome coordinates
# This only gets inactive ribosomes (not actively translating)
ribosomes = np.argwhere(pL==getIdx('ribosomeP'))
ribosomes.shape

In [ ]:
# Now we will check all ribosome bound states and add them to the total list of ribosome coordinates
# This might take a minute.
for pID in sN:
    if pID.startswith('RB_'):
        coords = np.argwhere(pL==getIdx(pID))
        if coords.shape[0]>0:
            ribosomes = np.concatenate((ribosomes, coords), axis=0)
ribosomes.shape

In [ ]:
traj = h5py.File('/home/zane/Models/human_rdme/Stem_Cell_Allen_ToyModel_1.lm')

In [ ]:
traj.keys()

In [ ]:
traj['Simulations']['0000001']['Lattice']['0000000000']

In [ ]:
traj['Parameters'].keys()

In [ ]:
traj['Model']['Diffusion']['LatticeSites']

In [ ]:
newDict = {}

In [ ]:
newDict['test'] = {'m':'L','d':'R'}

In [ ]:
newDict['test']['m']

In [ ]:
1300/60*4